In [2]:
# imports
import requests
from pprint import pprint
import pandas as pd
import os


# To load saved city bikes data
city_bikes = pd.read_pickle("city_bikes.pkl")

In [6]:
# To display all columns and prevent line wrapping
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)

# Foursquare

### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [38]:
# To convert the city bikes DataFrame to a list of dictionaries
bike_stations = city_bikes[['extra.name', 'latitude', 'longitude']].rename(
    columns={'extra.name': 'name'}
).to_dict(orient='records')

In [9]:
# Foursquare API key
API_KEY = os.environ.get('API_KEY')

# to set up url and API key
foursquare_url = "https://places-api.foursquare.com/places/search"

headers = {
    "accept": "application/json",
    "X-Places-Api-Version": "2025-06-17",
    "authorization": "Bearer API_KEY"
}

In [39]:
# To define the function that gets the nearby places
def get_nearby_places(lat, lon, radius=1000, limit=50):
    params = {
        "ll": f"{lat},{lon}",
        "radius": radius,
        "fsq_category_ids": "4d4b7105d754a06374d81259,4bf58dd8d48988d116941735",
        "limit": limit
    }

    response = requests.get(foursquare_url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return []

    results = response.json().get("results", [])
    places = []

    for place in results:
        places.append({
            "restaurant_bar_name": place.get("name"),
            "distance": place.get("distance"),
            "category": place["categories"][0]["name"] if place.get("categories") else None,
            "address": place["location"].get("formatted_address", "N/A"),
            "latitude": place.get('latitude'),
            "longitude": place.get('longitude')
        })

    return places

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [40]:
# To get the nearby restaurants and bars for each bike station in London
all_places = []

for station in bike_stations:
    places = get_nearby_places(station["latitude"], station["longitude"])
    for place in places:
        place["station_name"] = station["name"]
        place["station_latitude"] = station["latitude"]
        place["station_longitude"] = station["longitude"]
        all_places.append(place)

### Put your parsed results into a DataFrame

In [41]:
# the convert the results to pandas DataFrame
df_places = pd.DataFrame(all_places)

In [42]:
df_places

,restaurant_bar_name,distance,category,address,latitude,longitude,station_name,station_latitude,station_longitude
0,Putt In The Park Café,181,Pizzeria,"Putt in the Park Café, Battersea Park, Albert Bridge Rd, London SW11 4NJ",51.475883,-0.161511,"Prince of Wales Drive, Battersea Park",51.475154,-0.15917
1,The Lighthouse,365,Pub,"441 Battersea Park Rd, Battersea, Greater London, SW11 4LR",51.472635,-0.162558,"Prince of Wales Drive, Battersea Park",51.475154,-0.15917
2,Duke of Cambridge,519,Pub,"228 Battersea Bridge Rd (Banbury St), Battersea, Greater London, SW11 3AA",51.472987,-0.165825,"Prince of Wales Drive, Battersea Park",51.475154,-0.15917
3,Pear Tree Cafe,747,Café,"Battersea Park (Carriage Dr E), Battersea, Greater London, SW11 4NJ",51.479719,-0.151237,"Prince of Wales Drive, Battersea Park",51.475154,-0.15917
4,Antipasto & Pasta,587,Italian Restaurant,"511 Battersea Park Road, London, Greater London, SW11 3BW",51.471937,-0.165910,"Prince of Wales Drive, Battersea Park",51.475154,-0.15917
...,...,...,...,...,...,...,...,...,...
39803,The Folly,501,Cocktail Bar,"41 Gracechurch St (at Eastcheap), London, Greater London, EC3V 0BT",51.511035,-0.085788,"St. Mary Axe, Aldgate",51.514225,-0.08066
39804,Pret a Manger,464,Sandwich Spot,"1 Great Tower Street, London, Greater London, EC3R 5AA",51.510209,-0.082516,"St. Mary Axe, Aldgate",51.514225,-0.08066
39805,The Sugar Loaf,866,Pub,"65 Cannon St, London, Greater London, EC4N 5AA",51.512198,-0.092760,"St. Mary Axe, Aldgate",51.514225,-0.08066
39806,Long Arm Sports Pub & Brewery,936,Brewery,"20-26 Worship St, London, Greater London, EC2A 2DX",51.522090,-0.085528,"St. Mary Axe, Aldgate",51.514225,-0.08066


In [43]:
# Save dataframe
df_places.to_pickle("places.pkl")